In [1]:
# 카프카 라이브러리 설치
!pip install kafka-python

In [2]:
# 토픽을 읽을 파일을 생성
with open('new-topic.txt','w') as f:
    for i in range(1,1001):
        f.write(str(i)+'\n')

In [3]:
# 프로듀서 생성 (토픽으로 데이터를 전송)
from kafka import KafkaProducer
import json
import time
from csv import reader

In [4]:
class MessageProducer:
    broker,topic,producer = '','',None
    def __init__(self,broker,topic):
        self.broker = broker
        self.topic = topic
        params = {
            'bootstrap_servers' : self.broker,
            # 직렬화
            'value_serializer' : lambda x : json.dumps(x).encode('utf-8'),
            'acks':0,  # 클라이언트의 응답을 기다리지 않고 다음 메세지 전송(높은 전송속도를 보장, 메세지 전송을 보장하지 않을수 있음)
            'api_version':(2,5,0),
            'retries':3  # 재전송 횟수
        }
        # asks:1(default) 속도는 보통, 유실 가능성이 있음
        # asks:all or -1 속도는 가장 느림, 메세지 전달 손실 가능성

        self.producer = KafkaProducer(**params)
    def send_message(self, msg):
        try:
            future =  self.producer.send(self.topic, msg)
            self.producer.flush()  # 메세지 버퍼를 비워줌
            future.get(timeout=60)  # 60 초간 대기
            return {'status_code':200, 'error':None}
        except Exception as e:
            print('error:',e)
            return e

In [6]:
#브로커와 토픽명을 지정
broker = 'localhost:9092'
topic = 'test'
message_producer = MessageProducer(broker,topic)
with open('new-topic.txt','r',encoding='utf-8') as f:
    for data in f:
        res = message_producer.send_message(data)